In [1]:
!pip install torchaudio
!pip install transformers
!pip install datasets
!pip install lang_trans
!pip install arabic_reshaper
!pip install python-bidi
!pip install pydub
!pip install tnkeeh
!pip install soundfile
!pip install jiwer==2.2.0
!pip install PyArabic

     |████████████████████████████████| 3.1 MB 5.2 MB/s 
     |████████████████████████████████| 3.3 MB 47.5 MB/s 
     |████████████████████████████████| 61 kB 486 kB/s 
     |████████████████████████████████| 895 kB 46.1 MB/s 
     |████████████████████████████████| 596 kB 51.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 298 kB 5.2 MB/s 
     |████████████████████████████████| 1.1 MB 44.2 MB/s 
     |████████████████████████████████| 132 kB 54.9 MB/s 
     |████████████████████████████████| 243 kB 48.8 MB/s 
     |████████████████████████████████| 271 kB 35.2 MB/s 
     |████████████████████████████████| 160 kB 54.6 MB/s 
     |████████████████████████████████| 192 kB 60.6 MB/s 
  Created wheel for lang-trans: filename=lang_trans-0.6.0-py3-none-any.whl size=6345 sha256=5110f8495ff8b07d7e2a0e74479b555e3def48d0323ef4af42b747643994359b
  

In [2]:
import os
import torch
import jiwer
import torchaudio
import numpy as np
import pandas as pd
import tnkeeh as tn
import seaborn as sns
import soundfile as sf
import arabic_reshaper
from pyarabic import araby
from pydub import AudioSegment
import matplotlib.pyplot as plt
from pydub.utils import mediainfo
from bidi.algorithm import get_display
from lang_trans.arabic import buckwalter
from datasets import load_dataset, Dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
% matplotlib inline

In [3]:
!cp -r /content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/All_poems.zip .

In [4]:
if os.path.exists('dataset'):
  if len(os.listdir('dataset')) == 0:
    os.system('unzip All_poems.zip -d dataset')
else:
  os.system('unzip All_poems.zip -d dataset')

In [5]:
!mkdir -p dataset_wav

In [6]:
metadata_test_path = '/content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/testset.csv'
dataset_folder = 'dataset'
dataset_wav_folder = 'dataset_wav'

In [7]:
metadata = pd.read_csv(metadata_test_path)
metadata

,Unnamed: 0,No.,Utterance name,Script,Bahr,Source,Name when extacted,Original name in document,Further info on Bahr,Name of document,Clean Script,File Exist
0,1872,1883,P-1883.wma,غَيْرَ أَنِّي بِالجَوَى أَعْرِفُها وَهْيَ...,الرمل,NaN,ÛíÑÃäí.wma,غيرأني.wma,NaN,NaN,غير أني بالجوى أعرفها وهي أيضا بالجوى تعرفني,True
1,1290,1296,P-1296.wma,إِلى القُطِع وَما وَلّا ه مِن سَهلٍ وَمِ...,الهزج,NaN,Åáì ÇáÞØÚ.wma,إلى القطع.wma,NaN,NaN,إلى القطع وما ولا ه من سهل ومن وعر,True
2,2567,2601,P-2601.aac,إذا ابتُليتَ فصبرا فالعُسْر يُعقِب يُسرا,المجتث,التذكرة الحمدونية (8/ 68),36.aac,36.aac,NaN,NaN,إذا ابتليت فصبرا فالعسر يعقب يسرا,True
3,2696,2730,P-2730.opus,إِنْ كَانَ فِيْمَا أَقُوْلُ مِنْ كَرَمٍ ف...,المنسرح,الدر الفريد وبيت القصيد (4/ 190),75.opus,75.opus,NaN,NaN,إن كان فيما أقول من كرم فيك مزيد فزادك الله,True
4,1083,1088,P-1088.m4a,يُعْمِلُ البازِلَ المُجِدَّة بالرَّحْـ ـل...,الخفيف,المفضليات تحقيق أحمد شاكر وعبد السلام هارون صـ...,È 10 Þ 3.m4a,ب 10 ق 3.m4a,NaN,NaN,يعمل البازل المجدة بالرح ل تشكى النجاد بع...,True
...,...,...,...,...,...,...,...,...,...,...,...,...
362,559,560,P-560.m4a,الحَمدُ لِلَّهِ ذي البَنِيَّةِ إِذ أَمسَت...,الطويل,ديوان قيس بن الخطيم ص 175,ÇáÊÓÌíá (11).m4a,التسجيل (11).m4a,NaN,NaN,الحمد لله ذي البنية إذ أمست دحي قد أثخنت ...,True
363,239,240,P-240.m4a,نِعَمٌ للَهِ فينا رَبَّها وَصَنيعُ اللَهِ...,الرمل,ديوان سويد اليشكري 30,ÇáÊÓÌíá (85).m4a,التسجيل (85).m4a,NaN,NaN,نعم لله فينا ربها وصنيع الله والله صنع,True
364,2123,2157,P-2157.aac,قَدْ كان له قلبٌ كالطِّفْـ ـلِ يدُ الأحلا...,المتدارك,ديوان الشابي ص128,11.aac,11.aac,NaN,NaN,قد كان له قلب كالطف ل يد الأحلام تهدهده,True
365,1212,1218,P-1218.m4a,وكم قلت سوف يأتي إلى داره الغريب,المضارع,علم العروض والقافية (ص: 108),ÇáÊÓÌíá (28).m4a,التسجيل (28).m4a,NaN,NaN,وكم قلت سوف يأتي إلى داره الغريب,True


In [8]:
sample_rates = set()
for file_path in metadata['Utterance name']:
  complete_path = f'{dataset_folder}/{file_path}'
  complete_wav_path = f'{dataset_wav_folder}/{file_path}'
  # os.system(f'ffmpeg -i {complete_path} {complete_wav_path}')
  audio = AudioSegment.from_file(complete_path)
  sample_rates.add(audio.frame_rate)
  audio.export(f'{dataset_wav_folder}/{file_path}', format='wav')
sample_rates

{44100, 48000}

In [9]:
dataset = Dataset.from_pandas(metadata)
dataset

Dataset({
    features: ['Unnamed: 0', 'No.', 'Utterance name', 'Script', 'Bahr', 'Source', ' Name when extacted', 'Original name in document', 'Further info on Bahr', 'Name of document', 'Clean Script', 'File Exist'],
    num_rows: 367
})

In [10]:
resamplers = {  # The dataset contains all the uncommented sample rates
    48000: torchaudio.transforms.Resample(48000, 16000),
    44100: torchaudio.transforms.Resample(44100, 16000),
    # 32000: torchaudio.transforms.Resample(32000, 16000),
}

In [11]:
def prepare_example(example):
  complete_path = f'{dataset_wav_folder}/{example["Utterance name"]}'
  speech, sampling_rate = torchaudio.load(complete_path)
  speech = speech[0]
  example["speech"] = resamplers[sampling_rate](speech).squeeze().numpy()
  return example

In [12]:
def predict(batch):
    inputs = processor(batch["speech"], sampling_rate=16000, return_tensors="pt", padding=True)
    with torch.no_grad():
        predicted = torch.argmax(model(inputs.input_values.to('cuda')).logits, dim=-1)
    predicted[predicted == -100] = processor.tokenizer.pad_token_id  # see fine-tuning script
    batch["predicted"] = processor.tokenizer.batch_decode(predicted)
    return batch

In [13]:
from jiwer import AbstractTransform

class RemoveDiacritics(AbstractTransform):
  def process_string(self, s: str):
    return araby.strip_tashkeel(araby.strip_tatweel(s))

class TransliterateBukwalterToArabic(AbstractTransform):
  def process_string(self, s: str):
    return buckwalter.untrans(s)

In [14]:
# dataset1 = Dataset.from_dict(dataset[:2])

# Try on Elgiesh model

In [15]:
default_jiwer_transformations = [
    # default transformation below
    jiwer.RemoveMultipleSpaces(),
    jiwer.Strip(),
    jiwer.SentencesToListOfWords(),
    jiwer.RemoveEmptyStrings(),
]
processor = Wav2Vec2Processor.from_pretrained("elgeish/wav2vec2-large-xlsr-53-arabic")
model = Wav2Vec2ForCTC.from_pretrained("elgeish/wav2vec2-large-xlsr-53-arabic").eval()
model.to('cuda')
dataset = dataset.map(prepare_example)
predictions = dataset.map(predict, batched=True, batch_size=1, remove_columns=["speech"])
""" compute measers: on bukwalter, consider diacritics"""
metrics = jiwer.compute_measures(
  truth=[buckwalter.trans(s) for s in dataset["Script"]],  # Buckwalter transliteration
  hypothesis=predictions["predicted"],
  truth_transform=jiwer.Compose(default_jiwer_transformations),
  hypothesis_transform=jiwer.Compose(default_jiwer_transformations),
)
print(f"wer for dataset WITH diacritics on bukwalter: {metrics['wer']:.2%}. Full metrics:{metrics}")
""" compute measers: on bukwalter, remove diacritics"""
metrics = jiwer.compute_measures(
  # use Clean Script this time
  truth=[buckwalter.trans(s) for s in dataset["Clean Script"]],  # Buckwalter transliteration
  hypothesis=predictions["predicted"],
  truth_transform=jiwer.Compose([jiwer.SubstituteRegexes({
      r'[auiFNKo\~_،؟»\?;:\-,\.؛«!"]': "", "\u06D6": "",
      r"[\|\{]": "A", "p": "h", "ک": "k", "ی": "y"})]+default_jiwer_transformations),
  hypothesis_transform=jiwer.Compose([jiwer.SubstituteRegexes({
      r'[auiFNKo\~_،؟»\?;:\-,\.؛«!"]': "", "\u06D6": "",
      r"[\|\{]": "A", "p": "h", "ک": "k", "ی": "y"})]+default_jiwer_transformations),
)
print(f"wer for dataset WITHOUT diacritics on bukwalter: {metrics['wer']:.2%}. Full metrics:{metrics}")
""" compute measers: on arabic script, consider diacritics"""
metrics = jiwer.compute_measures(
  truth=dataset["Script"],  # Buckwalter transliteration
  hypothesis=predictions["predicted"],
  truth_transform=jiwer.Compose(default_jiwer_transformations),
  hypothesis_transform=jiwer.Compose([TransliterateBukwalterToArabic()]+default_jiwer_transformations),
)
print(f"wer for dataset WITH diacritics on arabic script: {metrics['wer']:.2%}. Full metrics:{metrics}")
""" compute measers: on arabic script, remove diacritics"""
metrics = jiwer.compute_measures(
  # use Clean Script this time
  truth=dataset["Clean Script"],  # Buckwalter transliteration
  hypothesis=predictions["predicted"],
  truth_transform=jiwer.Compose(default_jiwer_transformations),
  hypothesis_transform=jiwer.Compose([jiwer.SubstituteRegexes({
      r'[auiFNKo\~_،؟»\?;:\-,\.؛«!"]': "", "\u06D6": "",
      r"[\|\{]": "A", "p": "h", "ک": "k", "ی": "y"}),TransliterateBukwalterToArabic()]+default_jiwer_transformations),
)
print(f"wer for dataset WITHOUT diacritics on arabic script: {metrics['wer']:.2%}. Full metrics:{metrics}")
print('samples:')
print('predicted bukwalter:',predictions["predicted"][:5])
print('predicted:',[buckwalter.untrans(s) for s in predictions["predicted"][:5]])
print('clean script bukwalter:',[buckwalter.trans(s) for s in dataset["Clean Script"][:5]])
print('clean script:',dataset["Clean Script"][:5])
print('script bukwalter:',[buckwalter.trans(s) for s in dataset["Script"][:5]])
print('script:',dataset["Script"][:5])

Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/507 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/303 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

  0%|          | 0/367 [00:00<?, ?ex/s]

  0%|          | 0/367 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  tensor = as_tensor(value)


wer for dataset WITH diacritics on bukwalter: 88.81%. Full metrics:{'wer': 0.8880989180834621, 'mer': 0.884, 'wil': 0.9860657228389733, 'wip': 0.013934277161026692, 'hits': 377, 'substitutions': 2761, 'deletions': 97, 'insertions': 15}
wer for dataset WITHOUT diacritics on bukwalter: 42.57%. Full metrics:{'wer': 0.42569659442724456, 'mer': 0.41818734793187345, 'wil': 0.6406617512045631, 'wip': 0.3593382487954368, 'hits': 1913, 'substitutions': 1182, 'deletions': 135, 'insertions': 58}
wer for dataset WITH diacritics on arabic script: 88.81%. Full metrics:{'wer': 0.8880989180834621, 'mer': 0.884, 'wil': 0.9860657228389733, 'wip': 0.013934277161026692, 'hits': 377, 'substitutions': 2761, 'deletions': 97, 'insertions': 15}
wer for dataset WITHOUT diacritics on arabic script: 44.98%. Full metrics:{'wer': 0.4498452012383901, 'mer': 0.44190997566909973, 'wil': 0.6693674214640536, 'wip': 0.3306325785359464, 'hits': 1835, 'substitutions': 1260, 'deletions': 135, 'insertions': 58}
samples:
pred

# Try on sina model

In [16]:
dataset = Dataset.from_pandas(metadata)
dataset

Dataset({
    features: ['Unnamed: 0', 'No.', 'Utterance name', 'Script', 'Bahr', 'Source', ' Name when extacted', 'Original name in document', 'Further info on Bahr', 'Name of document', 'Clean Script', 'File Exist'],
    num_rows: 367
})

In [17]:
default_jiwer_transformations = [
    # default transformation below
    jiwer.RemoveMultipleSpaces(),
    jiwer.Strip(),
    jiwer.SentencesToListOfWords(),
    jiwer.RemoveEmptyStrings(),
]
processor = Wav2Vec2Processor.from_pretrained("bakrianoo/sinai-voice-ar-stt")
model = Wav2Vec2ForCTC.from_pretrained("bakrianoo/sinai-voice-ar-stt").eval()
model.to('cuda')
dataset = dataset.map(prepare_example)
predictions = dataset.map(predict, batched=True, batch_size=1, remove_columns=["speech"])
""" compute measers: on bukwalter, consider diacritics"""
metrics = jiwer.compute_measures(
  truth=[buckwalter.trans(s) for s in dataset["Script"]],  # Buckwalter transliteration
  hypothesis=predictions["predicted"],
  truth_transform=jiwer.Compose(default_jiwer_transformations),
  hypothesis_transform=jiwer.Compose(default_jiwer_transformations),
)
print(f"wer for dataset WITH diacritics on bukwalter: {metrics['wer']:.2%}. Full metrics:{metrics}")
""" compute measers: on bukwalter, remove diacritics"""
metrics = jiwer.compute_measures(
  # use Clean Script this time
  truth=[buckwalter.trans(s) for s in dataset["Clean Script"]],  # Buckwalter transliteration
  hypothesis=predictions["predicted"],
  truth_transform=jiwer.Compose([jiwer.SubstituteRegexes({
      r'[auiFNKo\~_،؟»\?;:\-,\.؛«!"]': "", "\u06D6": "",
      r"[\|\{]": "A", "p": "h", "ک": "k", "ی": "y"})]+default_jiwer_transformations),
  hypothesis_transform=jiwer.Compose([jiwer.SubstituteRegexes({
      r'[auiFNKo\~_،؟»\?;:\-,\.؛«!"]': "", "\u06D6": "",
      r"[\|\{]": "A", "p": "h", "ک": "k", "ی": "y"})]+default_jiwer_transformations),
)
print(f"wer for dataset WITHOUT diacritics on bukwalter: {metrics['wer']:.2%}. Full metrics:{metrics}")
""" compute measers: on arabic script, consider diacritics"""
metrics = jiwer.compute_measures(
  truth=dataset["Script"],  # Buckwalter transliteration
  hypothesis=predictions["predicted"],
  truth_transform=jiwer.Compose(default_jiwer_transformations),
  hypothesis_transform=jiwer.Compose([TransliterateBukwalterToArabic()]+default_jiwer_transformations),
)
print(f"wer for dataset WITH diacritics on arabic script: {metrics['wer']:.2%}. Full metrics:{metrics}")
""" compute measers: on arabic script, remove diacritics"""
metrics = jiwer.compute_measures(
  # use Clean Script this time
  truth=dataset["Clean Script"],  # Buckwalter transliteration
  hypothesis=predictions["predicted"],
  truth_transform=jiwer.Compose(default_jiwer_transformations),
  hypothesis_transform=jiwer.Compose([jiwer.SubstituteRegexes({
      r'[auiFNKo\~_،؟»\?;:\-,\.؛«!"]': "", "\u06D6": "",
      r"[\|\{]": "A", "p": "h", "ک": "k", "ی": "y"}),TransliterateBukwalterToArabic()]+default_jiwer_transformations),
)
print(f"wer for dataset WITHOUT diacritics on arabic script: {metrics['wer']:.2%}. Full metrics:{metrics}")
print('samples:')
print('predicted bukwalter:',predictions["predicted"][:5])
print('predicted:',[buckwalter.untrans(s) for s in predictions["predicted"][:5]])
print('clean script bukwalter:',[buckwalter.trans(s) for s in dataset["Clean Script"][:5]])
print('clean script:',dataset["Clean Script"][:5])
print('script bukwalter:',[buckwalter.trans(s) for s in dataset["Script"][:5]])
print('script:',dataset["Script"][:5])

Downloading:   0%|          | 0.00/158 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/457 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/138 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:341: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

  0%|          | 0/367 [00:00<?, ?ex/s]

  0%|          | 0/367 [00:00<?, ?ba/s]

wer for dataset WITH diacritics on bukwalter: 100.00%. Full metrics:{'wer': 1.0, 'mer': 1.0, 'wil': 1.0, 'wip': 0.0, 'hits': 0, 'substitutions': 3146, 'deletions': 89, 'insertions': 0}
wer for dataset WITHOUT diacritics on bukwalter: 100.00%. Full metrics:{'wer': 1.0, 'mer': 1.0, 'wil': 1.0, 'wip': 0.0, 'hits': 0, 'substitutions': 3146, 'deletions': 84, 'insertions': 0}
wer for dataset WITH diacritics on arabic script: 92.30%. Full metrics:{'wer': 0.9230293663060278, 'mer': 0.9201848998459168, 'wil': 0.993408769114825, 'wip': 0.006591230885174964, 'hits': 259, 'substitutions': 2877, 'deletions': 99, 'insertions': 10}
wer for dataset WITHOUT diacritics on arabic script: 33.31%. Full metrics:{'wer': 0.3331269349845201, 'mer': 0.32895139101192294, 'wil': 0.5258586755209329, 'wip': 0.47414132447906715, 'hits': 2195, 'substitutions': 910, 'deletions': 125, 'insertions': 41}
samples:
predicted bukwalter: ['غير أني بالجواء حرفها وهي أيضا بالجوىت عرفني', 'إلى القطع وماولاه من سهل ومن وعري', 'إ